In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'
import rosbag
import matplotlib.pyplot as plt
import numpy as np
from math import *
from scipy.interpolate import interp1d
from transformations import * 
plt.rc('figure', figsize=(10,5))
#plt.rc('figure', figsize=(20,15))

def quat2eulers(w, x, y ,z):
    r = atan2(2 * (w * x + y * z),
                    1 - 2 * (x * x + y * y))
    p = asin(2 * (w * y - z * x))
    y = atan2(2 * (w * z + x * y), 1 - 2 * (y * y + z * z))
    return y, p, r

def RMSE(predictions, targets):
    return np.sqrt(np.mean((predictions-targets)**2))

def read_pose_swarm_fused(bag, topic, _id):
    pos = []
    ypr = []
    ts = []
    print(f"Read poses from topic {topic}")
    for topic, msg, t in bag.read_messages(topics=[topic]):
        for i in range(len(msg.ids)):
            _i = msg.ids[i]
            if _i == _id:
                ts.append(msg.header.stamp.to_sec())
                pos.append([msg.local_drone_position[i].x, msg.local_drone_position[i].y, msg.local_drone_position[i].z])
                ypr.append([msg.local_drone_yaw[i], 0, 0])
    ret = {
        "t": np.array(ts),
        "pos": np.array(pos),
       "pos_func": interp1d(ts, pos,axis=0),
        "ypr": np.array(ypr),
        "ypr_func": interp1d(ts, ypr,axis=0)
    }
    return ret
    
def read_pose(bag, topic):
    pos = []
    ypr = []
    ts = []
    print(f"Read poses from topic {topic}")
    for topic, msg, t in bag.read_messages(topics=[topic]):
        p = msg.pose.position
        q = msg.pose.orientation
        pos.append([p.x, p.y, p.z])
        y, p, r = quat2eulers(q.w, q.x, q.y, q.z)
        ypr.append([y, p, r])
        ts.append(msg.header.stamp.to_sec())
    ret = {
        "t": np.array(ts),
        "pos": np.array(pos),
       "pos_func": interp1d(ts, pos,axis=0),
        "ypr": np.array(ypr),
        "ypr_func": interp1d(ts, ypr,axis=0)
    }
    return ret

def read_loops(bag, topic="/swarm_loop/loop_connection"):
    loops = []
    for topic, msg, t in bag.read_messages(topics=[topic]):
        loop = {
            "ts_a": msg.ts_a.to_sec(),
            "ts_b": msg.ts_b.to_sec(),
            "id_a":msg.id_a,
            "id_b":msg.id_b,
            "dpos":np.array([msg.dpos.x, msg.dpos.y, msg.dpos.z]),
            "dyaw":msg.dyaw
        }
        loops.append(loop)
    return loops
    
def bag_read(bagname, nodes = [1, 2], is_pc=False):
    bag = rosbag.Bag(bagname)
    poses = {}
    loops = read_loops(bag, "/swarm_loop/loop_connection")
    poses_fused = {}
    for i in nodes:
        poses[i] =  read_pose(bag, f"/SwarmNode{i}/pose")
        if is_pc:
            poses_fused[i] = read_pose_swarm_fused(bag, "/swarm_drones/swarm_drone_fused_pc", i)
        else:
            poses_fused[i] = read_pose_swarm_fused(bag, "/swarm_drones/swarm_drone_fused", i)
    bag.close()
    return poses, loops, poses_fused
    


Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
nodes = [1, 2]
#poses, loops , poses_fused =bag_read("data/swarm_local_2020-12-05-23-31-24-flyzone.bag", nodes) ##This bag has out of flyzone part
#poses, loops , poses_fused =bag_read("data/swarm_local_2020-12-05-23-31-24.bag", nodes) ##This bag has out of flyzone part
# poses, loops, poses_fused = bag_read("data/swarm_local_2020-12-07-17-09-41.bag", nodes)
#poses, loops, poses_fused = bag_read("data/swarm_local_2020-12-07-17-34-57.bag", nodes, False)
#poses, loops, poses_fused = bag_read("data/swarm_local_2020-12-07-17-34-57.bag", nodes)

poses, loops, poses_fused = bag_read("data/swarm_local_pc.bag", nodes, True)


Read poses from topic /SwarmNode1/pose
Read poses from topic /swarm_drones/swarm_drone_fused_pc
Read poses from topic /SwarmNode2/pose
Read poses from topic /swarm_drones/swarm_drone_fused_pc


In [3]:
fig = plt.figure("Ground Truth3d")
ax = fig.add_subplot(111, projection='3d')
ax = fig.gca(projection='3d')
main_id = 1
fused_offset = poses[main_id]["pos"][0] - poses_fused[main_id]["pos"][0]

#Plot Ground Truth3d
for i in nodes:
    ax.plot(poses[i]["pos"][:,0], poses[i]["pos"][:,1],poses[i]["pos"][:,2], label=f"Vicon Traj{i}")
    #ax.plot(poses_fused[i]["pos"][:,0] + fused_offset[0], poses_fused[i]["pos"][:,1] + fused_offset[1],poses_fused[i]["pos"][:,2] + fused_offset[2], label=f"Fused Traj{i}")

#Plot Loops
quivers = []
for loop in loops:
    posa_gt = poses[loop["id_a"]]["pos_func"](loop["ts_a"])
    posb_gt = poses[loop["id_b"]]["pos_func"](loop["ts_b"])
    quivers.append([posa_gt[0], posa_gt[1], posa_gt[2], posb_gt[0]-posa_gt[0], posb_gt[1]-posa_gt[1], posb_gt[2]-posa_gt[2]])
quivers = np.array(quivers)
c = np.arctan2(quivers[:,4], quivers[:,3])
c = (c.ravel() - c.min()) / c.ptp()
c = np.concatenate((c, np.repeat(c, 2)))
c = plt.cm.hsv(c)
ax.quiver(quivers[:,0], quivers[:,1], quivers[:,2], quivers[:,3], quivers[:,4], quivers[:,5], arrow_length_ratio=0.1, colors = c)

plt.legend()
plt.show()


#Plot Fused Vs GT 3D
fig = plt.figure("Fused Vs GT 3D")
for i in nodes:
    ax = fig.add_subplot(120 + i, projection='3d')
    ax = fig.gca(projection='3d')
    ax.plot(poses[i]["pos"][:,0], poses[i]["pos"][:,1],poses[i]["pos"][:,2], label=f"Vicon Traj{i}")
    ax.plot(poses_fused[i]["pos"][:,0] + fused_offset[0], poses_fused[i]["pos"][:,1] + fused_offset[1],poses_fused[i]["pos"][:,2] + fused_offset[2], label=f"Fused Traj{i}")
    plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
#Plot Fused Vs GT 1D
fig = plt.figure("Fused Vs GT 1D")
ax1, ax2, ax3 = fig.subplots(3, 1)

for i in nodes:
    t_ = poses_fused[i]["t"]
    pos_gt =  poses[i]["pos_func"](poses_fused[i]["t"])
    pos_fused = poses_fused[i]["pos"]
    _i = str(i) 
    
    ax1.plot(t_, pos_gt[:,0], label="$x_{gt}^" + _i + "$")
    ax1.plot(t_, pos_fused[:,0], label="$x_{fused}^" + _i + "$")

    ax2.plot(t_, pos_gt[:,1], label="$y_{gt}^" + _i + "$")
    ax2.plot(t_, pos_fused[:,1], label="$y_{fused}^" + _i + "$")
    
    ax3.plot(t_, pos_gt[:,2], label="$z_{gt}^" + _i + "$")
    ax3.plot(t_, pos_fused[:,2], label="$z_{fused}^" + _i + "$")

ax1.legend()
ax2.legend()
ax3.legend()
ax1.grid()
ax2.grid()
ax3.grid()
plt.show()

#Plot Fused Vs GT absolute error
fig = plt.figure("Fused Absolute Error")
ax1, ax2, ax3 = fig.subplots(3, 1)
for i in nodes:
    t_ = poses_fused[i]["t"]
    t0 =t_[0]
    pos_gt =  poses[i]["pos_func"](poses_fused[i]["t"])
    pos_fused = poses_fused[i]["pos"]
    _i = str(i) 
    rmse_x = RMSE(pos_gt[:,0] , pos_fused[:,0])
    rmse_y = RMSE(pos_gt[:,1] , pos_fused[:,1])
    rmse_z = RMSE(pos_gt[:,2] , pos_fused[:,2])
    label = f"$errx_{i}$ RMSE{i}:{rmse_x:3.3f}"
    ax1.plot(t_, pos_gt[:,0]  - pos_fused[:,0], label=label)
    
    label = f"$erry_{i}$ RMSE{i}:{rmse_y:3.3f}"
    ax2.plot(t_, pos_gt[:,1]  - pos_fused[:,1], label=label)
    
    label = f"$erry_{i}$ RMSE{i}:{rmse_z:3.3f}"
    ax3.plot(t_,  pos_gt[:,1]  - pos_fused[:,1], label=label)
    
    print(f"RMSE {i} is {rmse_x:3.3f},{rmse_y:3.3f},{rmse_z:3.3f}")

ax1.legend()
ax2.legend()
ax3.legend()
ax1.grid()
ax2.grid()
ax3.grid()
plt.show()

plt.figure("Yaw")
for i in nodes:
    t_ = poses_fused[i]["t"]
    yaw_gt =  poses[i]["ypr_func"](poses_fused[i]["t"])
    yaw_fused = poses_fused[i]["ypr"]
    plt.plot(t_,  yaw_gt[:,0], label=f"$\psi_GT{i}$")
    plt.plot(t_,  yaw_fused[:,0], label=f"$\psi_Fused{i}$")
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RMSE 1 is 0.377,0.872,0.088
RMSE 2 is 0.237,1.067,0.168


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
_loops_data = []
dpos_loops = []
dpos_gts = []
dpos_gt_norms= []
dpos_loop_norms= []
dpos_errs = []
dpos_errs_norm = []
posa_gts = []
ts_a = []
dyaws = []
dyaw_gts = []
yawa_gts = []
yawb_gts = []
dyaw_errs = []
print("Total loops", len(loops))
for loop in loops:
    posa_gt = poses[loop["id_a"]]["pos_func"](loop["ts_a"])
    posb_gt = poses[loop["id_b"]]["pos_func"](loop["ts_b"])
    yawa_gt = poses[loop["id_a"]]["ypr_func"](loop["ts_a"])[0]
    yawb_gt = poses[loop["id_b"]]["ypr_func"](loop["ts_b"])[0]
    dyaw_gt = yawb_gt - yawa_gt
    dpos_gt = np.array(posb_gt - posa_gt)
    dpos_gt = np.transpose([dpos_gt])
    Re = rotation_matrix(-yawa_gt, [0, 0, 1])[0:3, 0:3]
    dpos_gt = np.dot(Re, dpos_gt)
    dpos_gt = dpos_gt.flatten()
    dpos_loop = np.array(loop["dpos"])
    _loops_data.append({
        "dpos_loop": dpos_loop,
        "dpos_gt": dpos_gt,
        "dpos_err": dpos_gt - dpos_loop
        })
    dpos_loops.append(dpos_loop)
    dpos_gts.append(dpos_gt)
    dpos_errs.append(dpos_gt - dpos_loop)    
    dpos_gt_norms.append(np.linalg.norm(dpos_gt))
    dpos_loop_norms.append(np.linalg.norm(dpos_loop))
    dpos_errs_norm.append(np.linalg.norm(dpos_gt - dpos_loop))
    posa_gts.append(posa_gt)
    dyaws.append(loop["dyaw"])
    dyaw_gts.append(yawb_gt-yawa_gt)
    ts_a.append(loop["ts_a"])
    yawa_gts.append(yawa_gt)
    yawb_gts.append(yawb_gt)
    dyaw_errs.append(yawb_gt-yawa_gt-loop["dyaw"])
    
    if np.linalg.norm(dpos_gt - dpos_loop) > 1.0:
        print("Error", np.linalg.norm(dpos_gt - dpos_loop) , loop)
    
posa_gts = np.array(posa_gts)
dpos_errs = np.array(dpos_errs)
fig = plt.figure()

plt.subplot(311)
plt.plot(ts_a, dpos_errs_norm, '.', label="Loop Error")
plt.plot(ts_a, np.abs(dpos_errs[:,0]), '+', label="Loop Error X")
plt.plot(ts_a, np.abs(dpos_errs[:,1]), '+', label="Loop Error Y")
plt.plot(ts_a, dpos_errs[:,2], '+', label="Loop Error Z")
plt.title("Error Pos Loop vs Vicon")
plt.grid(which="both")
plt.legend()

plt.subplot(312)
plt.plot(ts_a, dyaws, '.', label="DYaw Gt")
plt.plot(ts_a, dyaw_gts, '+', label="DYaw Loop")
plt.plot(ts_a, yawa_gts, "*", label="Vicon Yaw A")
plt.plot(ts_a, np.abs(dyaw_errs), "x", label="DYaw Error")

#plt.plot(ts_a, yawb_gts, "*", label="Vicon Yaw B")
plt.title("Error Yaw Loop vs Vicon")
plt.grid(which="both")
plt.legend()


plt.subplot(313)
plt.plot(ts_a, posa_gts[:,0], '+', label="Vicon X")
plt.plot(ts_a, posa_gts[:,1], '+', label="Vicon Y")
plt.plot(ts_a, posa_gts[:,2], '+', label="Vicon Z")
plt.plot(poses[i]["t"], poses[i]["pos"][:,0], label="Vicon X")
plt.plot(poses[i]["t"], poses[i]["pos"][:,1], label="Vicon Y")
plt.plot(poses[i]["t"], poses[i]["pos"][:,2], label="Vicon Z")

plt.grid(which="both")
plt.legend()
plt.show()

plt.figure()
plt.subplot(141)
plt.hist(dpos_errs_norm, 5, density=True, facecolor='g', alpha=0.75)
plt.subplot(142)
plt.hist(dpos_errs[:,0], 5, density=True, facecolor='g', alpha=0.75)
plt.subplot(143)
plt.hist(dpos_errs[:,1], 5, density=True, facecolor='g', alpha=0.75)
plt.subplot(144)
plt.hist(dpos_errs[:,2], 5, density=True, facecolor='g', alpha=0.75)


print("Pos cov", np.cov(dpos_errs[:,0]), np.cov(dpos_errs[:,1]), np.cov(dpos_errs[:,2]) )
print("Yaw cov", np.cov(dyaw_errs))
plt.show()


# plt.figure()
# plt.subplot(211)
# plt.plot(dpos_gt_norms, dpos_errs_norm, 'o', label="GtDistance vs Error")
# plt.grid(which="both")
# plt.subplot(212)
# plt.plot(dpos_loop_norms, dpos_errs_norm, 'o', label="LoopDistance vs Error")
# plt.grid(which="both")
# plt.legend()
# plt.show()

Total loops 78
Error 4.176388161081254 {'ts_a': 1607333783.4896495, 'ts_b': 1607333784.014473, 'id_a': 2, 'id_b': 1, 'dpos': array([ 0.87191366,  3.40617199, -0.27344697]), 'dyaw': 0.1772447137852966}
Error 1.2904963567958225 {'ts_a': 1607333820.9396555, 'ts_b': 1607333721.968874, 'id_a': 2, 'id_b': 1, 'dpos': array([ 0.01956518, -0.24168934, -0.87646124]), 'dyaw': 0.094772837973626}
Error 1.955376236525415 {'ts_a': 1607333858.6896696, 'ts_b': 1607333721.968874, 'id_a': 2, 'id_b': 1, 'dpos': array([0., 0., 0.]), 'dyaw': 0.0}
Error 2.0760786407161564 {'ts_a': 1607333880.8896627, 'ts_b': 1607333721.968874, 'id_a': 2, 'id_b': 1, 'dpos': array([0., 0., 0.]), 'dyaw': 0.0}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pos cov 0.08672705896599142 0.33853914305915234 0.006549974672347518
Yaw cov 0.020961324630109177
